## Meter

In [50]:
# force geopandas to use shapely instead of pygeos
import os
os.environ['USE_PYGEOS'] = '0'

import numpy as np
import pandas as pd
from shapely import wkt
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px

In [4]:
meter_df = pd.read_csv('../data/Parking_Meters.csv')
meter_df.head()

,OBJECTID,PARKING_SPACE_ID,POST_ID,MS_PAY_STATION_ID,MS_SPACE_NUM,SENSOR_FLAG,ON_OFFSTREET_TYPE,OSP_ID,JURISDICTION,PM_DISTRICT_ID,...,NFC_KEY,SPT_CODE,COLLECTION_ROUTE_DESC,COLLECTION_SUBROUTE_DESC,shape,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,5262398,0,363-04151,-,0,N,ON,0.0,SFMTA,12.0,...,NaN,NaN,NaN,NaN,POINT (-122.408394 37.79895),106,106,6,3,6
1,5262423,0,916-00500,916-00002,50,N,OFF,916.0,SFMTA,7.0,...,5L4X,NaN,NaN,NaN,POINT (-122.43913 37.800167),17,17,4,6,13
2,5263054,0,700-03540,-,0,N,ON,0.0,SFMTA,16.0,...,2XKW,NaN,NaN,NaN,POINT (-122.422165 37.76667),37,37,3,5,20
3,5289824,0,563-01014,-,0,N,ON,0.0,SFMTA,13.0,...,NaN,NaN,NaN,NaN,POINT (-122.41383 37.780846),21,21,5,10,36
4,5263507,0,614-07280,-,0,N,ON,0.0,SFMTA,13.0,...,GK56,NaN,NaN,NaN,POINT (-122.41428 37.787613),50,50,6,3,21


In [11]:
meter_df['ACTIVE_METER_FLAG'].value_counts()

M    25670
U     5182
T     3643
P     1109
L       11
Name: ACTIVE_METER_FLAG, dtype: int64

In [8]:
meter_df['shape'] = meter_df['shape'].apply(wkt.loads)
meter_df = gpd.GeoDataFrame(meter_df, geometry='shape')
meter_df.crs = 'EPSG:4326'

In [14]:
active_meter_df = meter_df[(meter_df.ACTIVE_METER_FLAG == 'M') | (meter_df.ACTIVE_METER_FLAG == 'T')]
active_meter_df.shape

(29313, 45)

In [15]:
df = pd.read_parquet("../data/meter_data_2022.parquet")

In [20]:
df = pd.merge(df, active_meter_df, on='POST_ID')

In [21]:
df.head()

,TRANSMISSION_DATETIME,POST_ID,STREET_BLOCK,PAYMENT_TYPE,SESSION_START_DT,SESSION_END_DT,METER_EVENT_TYPE,GROSS_PAID_AMT,OBJECTID,PARKING_SPACE_ID,...,NFC_KEY,SPT_CODE,COLLECTION_ROUTE_DESC,COLLECTION_SUBROUTE_DESC,shape,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,222408714_4_01012022002410,831-02200,EMBARCADERO SOUTH 200,CASH,2022/01/01 12:24:10 AM,2022/01/01 12:24:10 AM,NS,1.50,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
1,222412313_4_01022022121822,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/02 12:18:22 PM,2022/01/02 01:18:22 PM,NS,8.00,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
2,222412993_4_01022022135210,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/02 01:52:10 PM,2022/01/02 03:52:17 PM,NS,14.49,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
3,222448944_4_01032022140304,831-02200,EMBARCADERO SOUTH 200,CREDIT CARD,2022/01/03 02:03:04 PM,2022/01/03 04:23:28 PM,NS,9.06,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8
4,832929888_9_01032022180700,831-02200,EMBARCADERO SOUTH 200,PAY BY CELL,2022/01/03 06:07:00 PM,2022/01/03 08:07:00 PM,NS,4.00,5272708,0,...,71ME,NaN,South Embarcadero,Embarcadero South/Howard/Steuart | Unit-300/Un...,POINT (-122.39102 37.79161),108,108,1,10,8


In [45]:
# use only a very limited number of columns for now

df = df[['POST_ID', 'TRANSMISSION_DATETIME', 'STREET_BLOCK', 'PAYMENT_TYPE',
    'SESSION_START_DT', 'SESSION_END_DT', 'METER_EVENT_TYPE',
    'GROSS_PAID_AMT', 'shape']]

In [46]:
one_post = df[df.POST_ID == '360-05250']

In [47]:
one_post.loc[:, 'SESSION_START_DT'] = pd.to_datetime(one_post['SESSION_START_DT'])
one_post.loc[:, 'SESSION_END_DT'] = pd.to_datetime(one_post['SESSION_END_DT'])

/var/folders/lr/597pxhmx2z78r4vysc_dvsth0000gn/T/ipykernel_41258/1276305083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_post.loc[:, 'SESSION_START_DT'] = pd.to_datetime(one_post['SESSION_START_DT'])
/var/folders/lr/597pxhmx2z78r4vysc_dvsth0000gn/T/ipykernel_41258/1276305083.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  one_post.loc[:, 'SESSION_START_DT'] = pd.to_datetime(one_post['SESSION_START_DT'])
/var/folders/lr/597pxhmx2z78r4vysc_dvsth0000gn/T/ipykernel_41258/1276305083.py:2: SettingWithCopyWarning: 
A val

In [49]:
one_post['date'] = one_post['SESSION_START_DT'].dt.date
one_date = one_post[one_post.date == pd.to_datetime('2022-01-28')]

/var/folders/lr/597pxhmx2z78r4vysc_dvsth0000gn/T/ipykernel_41258/2037813124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_post['date'] = one_post['SESSION_START_DT'].dt.date
/var/folders/lr/597pxhmx2z78r4vysc_dvsth0000gn/T/ipykernel_41258/2037813124.py:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  one_date = one_post[one_post.date == pd.to_datetime('2022-01-28')]


In [36]:
one_post.date.value_counts()

2022-01-28    32
2022-07-07    30
2022-12-02    30
2022-08-25    29
2022-09-03    29
              ..
2022-07-10     1
2022-05-15     1
2022-03-20     1
2022-12-29     1
2022-02-20     1
Name: date, Length: 317, dtype: int64

In [53]:
one_post

,POST_ID,TRANSMISSION_DATETIME,STREET_BLOCK,PAYMENT_TYPE,SESSION_START_DT,SESSION_END_DT,METER_EVENT_TYPE,GROSS_PAID_AMT,shape,date
467325,360-05250,222409320_4_01012022113609,CLEMENT ST 500,CASH,2022-01-01 11:36:09,2022-01-01 11:36:09,NS,0.15,POINT (-122.46503 37.78285),2022-01-01
467326,360-05250,222409409_4_01012022115811,CLEMENT ST 500,CASH,2022-01-01 11:58:11,2022-01-01 11:58:11,NS,0.25,POINT (-122.46503 37.78285),2022-01-01
467327,360-05250,222421195_4_01032022094613,CLEMENT ST 500,CASH,2022-01-03 09:46:13,2022-01-03 10:01:13,NS,0.50,POINT (-122.46503 37.78285),2022-01-03
467328,360-05250,222422538_4_01032022100307,CLEMENT ST 500,CREDIT CARD,2022-01-03 10:03:07,2022-01-03 10:18:07,NS,0.50,POINT (-122.46503 37.78285),2022-01-03
467329,360-05250,222424052_4_01032022101945,CLEMENT ST 500,CASH,2022-01-03 10:19:45,2022-01-03 10:34:45,NS,0.85,POINT (-122.46503 37.78285),2022-01-03
...,...,...,...,...,...,...,...,...,...,...
472193,360-05250,1041785889_9_12312022095234,CLEMENT ST 500,PAY BY CELL,2022-12-31 09:52:34,2022-12-31 10:07:34,NS,0.44,POINT (-122.46503 37.78285),2022-12-31
472194,360-05250,238182919_4_12312022140853,CLEMENT ST 500,CASH,2022-12-31 14:08:53,2022-12-31 14:23:53,NS,1.05,POINT (-122.46503 37.78285),2022-12-31
472195,360-05250,238184022_4_12312022143454,CLEMENT ST 500,CASH,2022-12-31 14:34:54,2022-12-31 14:45:49,NS,0.50,POINT (-122.46503 37.78285),2022-12-31
472196,360-05250,238184029_4_12312022143508,CLEMENT ST 500,CASH,2022-12-31 14:35:08,2022-12-31 14:50:08,NS,0.25,POINT (-122.46503 37.78285),2022-12-31


In [55]:
fig = px.timeline(one_date, x_start="SESSION_START_DT", x_end="SESSION_END_DT", y="METER_EVENT_TYPE")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

## Load Citation Data

In [56]:
citation_df = pd.read_csv("../data/data_2022_with_geom.csv")
citation_df.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom
0,950226616,04/20/2022 12:10:00 PM,TRC7.2.22,STR CLEAN,1318 FILBERT ST,CA,V507699,84.0,05/01/2022 12:00:00 AM,POINT (-122.42119800399996 37.800016984000024)
1,952126114,06/17/2022 04:12:00 PM,TRC7.2.20,RES/OT,1318 FILBERT ST,CA,BV44D97,96.0,07/17/2022 12:00:00 AM,POINT (-122.42119800399996 37.800016984000024)
2,950441903,04/23/2022 03:13:00 PM,V5200,NO PLATES,1318 FILBERT ST,CA,V507699,121.0,05/18/2022 12:00:00 AM,POINT (-122.42119800399996 37.800016984000024)
3,948460144,02/22/2022 11:54:00 AM,TRC7.2.20,RES/OT,1318 FILBERT ST,TX,HVT6737,96.0,05/23/2022 12:00:00 AM,POINT (-122.42119800399996 37.800016984000024)
4,948169644,02/17/2022 01:39:00 PM,TRC7.2.20,RES/OT,1318 FILBERT ST,OH,HQA6554,96.0,04/03/2022 12:00:00 AM,POINT (-122.42119800399996 37.800016984000024)


In [57]:
citation_df['geom'] = citation_df.geom.apply(wkt.loads)
citation_df = gpd.GeoDataFrame(citation_df, geometry='geom')
citation_df.crs = 'EPSG:4326'

In [62]:
# temporary filter

citation_df = citation_df[~citation_df['Citation Location'].isnull()]

In [58]:
citation_df['Citation Issued DateTime'] = pd.to_datetime(citation_df['Citation Issued DateTime'])

In [64]:
citation_df[citation_df['Citation Location'].str.contains('500 CLEMENT ST')]

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom
73897,950433035,2022-04-25 12:25:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,OR,917GMD,84.0,05/15/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73898,948417245,2022-02-28 12:28:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,8XZA904,84.0,03/21/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73899,951063654,2022-05-09 12:35:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,6YHL038,84.0,05/16/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73900,951069383,2022-05-09 12:36:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,5HZE558,84.0,07/20/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73901,951069372,2022-05-09 12:34:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,8PTC818,84.0,05/23/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73902,953417776,2022-07-25 12:30:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,6WZG298,87.0,08/15/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73903,953306830,2022-07-25 12:30:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,9BPK390,87.0,08/25/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73904,948369026,2022-02-28 12:28:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,8YCZ953,84.0,10/24/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73905,947139804,2022-01-10 12:27:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,5WRX038,84.0,01/30/2022 12:00:00 AM,POINT (-122.47558 37.78263)
73906,953306826,2022-07-25 12:30:00,TRC7.2.22,STR CLEAN,1500 CLEMENT ST,CA,45425R1,87.0,08/09/2022 12:00:00 AM,POINT (-122.47558 37.78263)
